In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
from pprint import pprint
import random
import pickle

## Helper functions

In [2]:
def generate_R_all(df, max_shp=6, shp_list=None):
    """
    input: df -- contains columns of trip id, on_id and off_id
           max_shp -- maximum number of shared paths for each trip
           shp_list -- list of number of sharable paths for each rider
    return: dictionary with {R_all: RiderSN, RiderEN, RShPath}
    """
    temp = df.copy()
    if shp_list is None:
        temp['sh_path'] = [random.randint(1, max_shp+1)
                           for _ in range(len(temp))]
    elif not (isinstance(shp_list, list) and all(isinstance(x, int) for x in shp_list)):
        raise TypeError("Invalid shp_list type!")
    elif len(shp_list) != len(df):
        raise ValueError(
            "Lenght of shp_list must equal to the lengh of input dataframe!")
    else:
        temp['sh_path'] = shp_list
    return {i: [row.on_id, row.off_id, row.sh_path] for i, row in temp.iterrows()}

In [3]:
def generate_D_all(num_d=6, fixed_route=None, od=None):
    """
    input: df -- dataframe include zone IDs
           num_d -- number of drivers
           fixed_route -- if None, shuttles are on-demand, else it represents
                          the number of itinerary loops for each driver
                          (list datatype)
           od -- list of od for each driver, i.e. od = [[o1, d1], [o2, d2],...]
    return: dictionary with {D_all: DriverSN, DriverEN, DShPath, DDuration}
    """
    d_sp = np.zeros((num_d,), int)

    if od is None:
        d_sn = random.sample([3, 150, 104, 17, 73, 159]*2, num_d)
        d_en = random.sample([3, 150, 104, 17, 73, 159]*2, num_d)
    else:
        d_sn, d_en = [x[0] for x in od], [x[1] for x in od]

    if fixed_route is not None:
        d_d = fixed_route
        return {i+1000: [item[0], item[1], item[2], item[3]] for i, item in enumerate(zip(d_sn, d_en, d_sp, d_d))}
    else:
        return {i+1000: [item[0], item[1], item[2]] for i, item in enumerate(zip(d_sn, d_en, d_sp))}

In [4]:
def generate_Links_Cost(T1, Nodes):
    """
    input: T1 -- set of time horizon
           Nodes -- node set
    return: list of tuples (t1, t2, n1, n2) with t1<=t2
    """

    # make sure one can only travel to its neighbour zones
    station = pickle.load(open(r"Data\temp\Station.p", "rb"))
    link = [(t1, t2, n1, n2) for t1 in T1
            for t2 in T1 if t1 <= t2
            for n1 in Nodes
            for n2 in station[n1]["neighbours"]]
    cost = [t2-t1 for (t1, t2, _, _) in link]
    return {l: c for l, c in zip(link, cost)}

In [5]:
def generate_Nodes_d(D_all, Nodes):
    """
    input: D_all -- driver set
           Nodes -- node set
    return: dictionary, feasible nodes for each driver
    """
    # By default each driver can reach any node
    return {i: Nodes for i in D_all}

In [6]:
def generate_Nodes_r(R_all, Nodes):
    """
    TO DO!
    """
    return None

In [7]:
def generate_Matches(R_all, D_all):
    """
    input: R_all -- rider set
           D_all -- driver set
    return: list of (rider, driver) tuples
    """
    return [(r, d) for r in R_all
            for d in D_all]

In [8]:
def generate_Links_d(T1, Nodes_d):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-8-46614a257007>, line 2)

## Global Parameters

In [1]:
NUM_DRIVERS = 5
FIXED_ROUTE = False
NUM_LOOP = [2]*NUM_DRIVERS  # No need if FIXED_ROUTE is False

## Set Initialization

In [10]:
df_zones = pd.read_csv(r"Data\zone_id.csv", index_col=["zone id"])
df_od = pd.read_csv(r"Data\pctrans_trip_od.csv")
df_count = pd.read_csv(r"Data\trip_count.csv")
df_distance = pd.read_csv(r"Data\census\station_distance.csv")
num_zones = len(df_zones)

# zone ID with non-zero inflow or outflow
nonzero_trip_zone = df_count.loc[(df_count.on_count != 0) | (
    df_count.off_count != 0)].index.values

In [11]:
# Node set
Nodes = nonzero_trip_zone  # only consider non-zero trip zones
# Time horizon
T1 = range(1, 36)
# Rider set
R_all, RiderSN, RiderEN, RShPath = gp.multidict(
    generate_R_all(df_od.iloc[0:300]))
# Driver set
if FIXED_ROUTE:
    D_all, DriverSN, DriverEN, DShPath, DDuration = gp.multidict(
        generate_D_all(fixed_route=NUM_LOOP))
else:
    D_all, DriverSN, DriverEN, DShPath = gp.multidict(generate_D_all())

# Link set
Links, Cost = gp.multidict(generate_Links_Cost(T1, Nodes))

# Node set for drivers
Nodes_d = generate_Nodes_d(D_all, Nodes)
# Node set for riders
Nodes_r = generate_Nodes_r(R_all, Nodes)  # TO DO!
# Match set for rider-driver pair
Matches = gp.tuplelist(generate_Matches(R_all, D_all))
# Link set for drivers
Link_d = generate_Links_d()

NameError: name 'generate_Links_d' is not defined

In [108]:
Matches[:10]

[(0, 1000),
 (0, 1001),
 (0, 1002),
 (0, 1003),
 (0, 1004),
 (0, 1005),
 (1, 1000),
 (1, 1001),
 (1, 1002),
 (1, 1003)]

## Parameter Initialization

In [25]:
# Dummy driver
D_DUMMY = D_all[-1]
# Vehicle capacity
VEH_CAP = 10

In [33]:
df_od.index.values

array([    0,     1,     2, ..., 11032, 11033, 11034], dtype=int64)